# Анализ схожести научных текстов с помощью методов естественной обработки языка и машинного обучения
## Береза Анастасия 
## Учебная группа о.ИЗДтс 23.2/Б3-22

## 1. Установка зависимостей


In [1]:
# При первом запуске раскомментировать строку ниже

# %pip install tensorflow kagglehub pandas numpy scikit-learn joblib tqdm psutil


## 2. Импорт библиотек


In [2]:
import kagglehub
import os
import shutil


f:\PDipPraktika\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 3. Загрузка датасета

In [3]:
path = kagglehub.dataset_download("ergkerg/russian-scientific-articles")

print("Path to dataset files:", path)

# копирование в папку проекта

destination_path = os.path.join(os.getcwd(), "russian-scientific-articles")
shutil.copytree(path, destination_path, dirs_exist_ok=True)

print("Файлы датасета перенесены в:", destination_path)

Path to dataset files: C:\Users\Basss\.cache\kagglehub\datasets\ergkerg\russian-scientific-articles\versions\1
Файлы датасета перенесены в: f:\PDipPraktika\russian-scientific-articles
